In [1]:
import pandas as pd
from datetime import timedelta
from IPython.display import HTML, display

In [2]:
date = pd.datetime.today() - timedelta(days=30)
org = 'jupyterhub'
repo = 'zero-to-jupyterhub-k8s'

In [3]:
# Parameters
org = "jupyterhub"
repo = "team-compass"
date = "2019-02-20"


In [4]:
# Definition of a stale issue/PR (how many days since last updated)
stale_days = 30
date_stale = pd.to_datetime(date) - timedelta(days=stale_days)

# Opened and Closed Issues and PRs

This report shows opened and closed issues / PRs over the last 30 days, with a
particular focus on items that haven't had a lot of discussion, or that have
been opened by people who aren't team-members. The goal of this page is to make
it easier to spot places where some input would be appreciated!

# Imports

In [5]:
import pandas as pd
from subprocess import run, PIPE
from watchtower import issues_
from glob import glob
import os.path as op
import shutil as sh
import sys
sys.path.append('../../scripts/')

## styling functions

In [6]:
def highlight_rows(val):
    """Highlights cells pertaining to non-members and w/ no comments"""
    disp = ''
    if val == 0 or val == "NONE":
        disp += 'background-color: #f7b7b7'
    elif val == "FIRST_TIME_CONTRIBUTOR":
        disp += 'background-color: red'
    return disp

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def print_html(msg):
    display(HTML('<h2 style="padding-left: 1em;">{}</h2>'.format(msg)))
    
pd.options.display.max_colwidth = 200

## Load data

In [7]:
# Issues
issues = issues_.load_issues(org, repo)
issues['user'] = issues['user'].map(lambda a: a['login'])

# New activity

## Opened issues

without responses in red

In [8]:
opened = issues.query('created_at > @date')
opened_issues = opened[opened['pull_request'].isna()]
opened_issues['closed'] = ~pd.isnull(opened_issues['closed_at'])

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
print_html('Opened issues: {}'.format(len(opened_issues)))

In [10]:
opened_issues[['user', 'author_association', 'comments', 'closed', 'html_url']]\
    .sort_values(['closed', 'comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,closed,html_url
Zsailer,MEMBER,0,False,https://github.com/jupyterhub/team-compass/issues/137
betatim,MEMBER,0,False,https://github.com/jupyterhub/team-compass/issues/136
betatim,MEMBER,1,False,https://github.com/jupyterhub/team-compass/issues/125
choldgraf,MEMBER,1,False,https://github.com/jupyterhub/team-compass/issues/135
willingc,MEMBER,2,False,https://github.com/jupyterhub/team-compass/issues/132
willingc,MEMBER,4,False,https://github.com/jupyterhub/team-compass/issues/127
choldgraf,MEMBER,4,False,https://github.com/jupyterhub/team-compass/issues/133
willingc,MEMBER,4,False,https://github.com/jupyterhub/team-compass/issues/131
choldgraf,MEMBER,5,False,https://github.com/jupyterhub/team-compass/issues/120
betatim,MEMBER,5,False,https://github.com/jupyterhub/team-compass/issues/134


## Closed issues


In [11]:
closed = issues.query('closed_at > @date')
closed_issues = closed[closed['pull_request'].isna()]

In [12]:
closed_issues[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
choldgraf,MEMBER,0,https://github.com/jupyterhub/team-compass/issues/30
betatim,MEMBER,8,https://github.com/jupyterhub/team-compass/issues/129
betatim,MEMBER,14,https://github.com/jupyterhub/team-compass/issues/111
Zsailer,MEMBER,14,https://github.com/jupyterhub/team-compass/issues/104
Zsailer,MEMBER,15,https://github.com/jupyterhub/team-compass/issues/122
betatim,MEMBER,25,https://github.com/jupyterhub/team-compass/issues/110


In [13]:
print_html('Closed issues: {}'.format(len(closed_issues)))

## Opened PRs

without responses in red

In [14]:
opened_prs = opened[~opened['pull_request'].isna()]
opened_prs['closed'] = ~pd.isnull(opened_prs['closed_at'])

closed_prs = closed[~closed['pull_request'].isna()]

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
print_html('Opened PRs: {}'.format(len(opened_prs)))

In [16]:
opened_prs[['user', 'author_association', 'comments', 'closed', 'html_url']]\
    .sort_values(['closed', 'comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,closed,html_url
Zsailer,MEMBER,1,True,https://github.com/jupyterhub/team-compass/pull/138
choldgraf,MEMBER,1,True,https://github.com/jupyterhub/team-compass/pull/128
yuvipanda,MEMBER,1,True,https://github.com/jupyterhub/team-compass/pull/123
Zsailer,MEMBER,1,True,https://github.com/jupyterhub/team-compass/pull/121
Zsailer,MEMBER,2,True,https://github.com/jupyterhub/team-compass/pull/139
choldgraf,MEMBER,2,True,https://github.com/jupyterhub/team-compass/pull/126


In [17]:
print_html('Closed PRs: {}'.format(len(closed_prs)))

In [18]:
closed_prs[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
Zsailer,MEMBER,1,https://github.com/jupyterhub/team-compass/pull/138
choldgraf,MEMBER,1,https://github.com/jupyterhub/team-compass/pull/128
yuvipanda,MEMBER,1,https://github.com/jupyterhub/team-compass/pull/123
Zsailer,MEMBER,1,https://github.com/jupyterhub/team-compass/pull/121
Zsailer,MEMBER,2,https://github.com/jupyterhub/team-compass/pull/139
choldgraf,MEMBER,2,https://github.com/jupyterhub/team-compass/pull/126


# State of repo

## Stale issues

In [19]:
stale_issues = issues.query('updated_at < @date_stale')
stale_issues = stale_issues[stale_issues['closed_at'].isnull()]

In [20]:
stale_issues[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
